This notebook shows how to run playoff scenarios prior to the end of regular season,
and identify how each team might win their division, whether they "control their
own destiny" etc. This example is scoped to the last two weeks of regular
season (weeks 17 and 18). If you're running the analysis after the end of
the season then these scores will be in the database, but you can clear them
and replace them with your own assumptions as shown below.

In the future I hope to do similar notebooks for wilcard slots and home-field advantage

In [9]:
from nfl import NFL
import pandas as pd
import numpy as np
import time

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

nfl = NFL().load()

In [10]:
# For reference, print conference standings not counting weeks 17 and 18
weeks = [17, 18]
nfl.clear(weeks)
nfl('NFC')

NFC
            div overall                    division                conference  \
                    win loss tie       pct      win loss tie   pct        win   
team                                                                            
PHI    NFC-East      11    4   0  0.733333        4    1   0   0.8          7   
DAL    NFC-East      10    5   0  0.666667        4    1   0   0.8          7   
NYG    NFC-East       5   10   0  0.333333        2    3   0   0.4          4   
WAS    NFC-East       4   11   0  0.266667        0    5   0   0.0          2   
DET   NFC-North      11    4   0  0.733333        3    2   0   0.6          7   
MIN   NFC-North       7    8   0  0.466667        2    2   0   0.5          6   
GB    NFC-North       7    8   0  0.466667        2    2   0   0.5          5   
CHI   NFC-North       6    9   0       0.4        2    3   0   0.4          5   
TB    NFC-South       8    7   0  0.533333        3    1   0  0.75          6   
ATL   NFC-South       7    8   0  0.466667        3    2   0   0.6          4   
NO    NFC-South       7    8   0  0.466667        2    2   0   0.5          4   
CAR   NFC-South       2   13   0  0.133333        1    4   0   0.2          1   
SF     NFC-West      11    4   0  0.733333        5    0   0   1.0          9   
STL    NFC-West       8    7   0  0.533333        4    1   0   0.8          6   
SEA    NFC-West       8    7   0  0.533333        1    4   0   0.2          6   
ARI    NFC-West       3   12   0       0.2        0    5   0   0.0          2   

                         
     loss tie       pct  
team                     
PHI     3   0       0.7  
DAL     3   0       0.7  
NYG     6   0       0.4  
WAS     8   0       0.2  
DET     3   0       0.7  
MIN     4   0       0.6  
GB      5   0       0.5  
CHI     5   0       0.5  
TB      4   0       0.6  
ATL     6   0       0.4  
NO      6   0       0.4  
CAR    10   0  0.090909  
SF      1   0       0.9  
STL     4   0       0.6  
SEA     5   0  0.545455  
ARI     8   0       0.2

In [11]:
# Specify the division and week range to run
div = 'NFC-East'
weeks = [17, 18]

nfl.reload()
nfl.clear(weeks)
nfl(div)

NFC-East
     overall                    division              
         win loss tie       pct      win loss tie  pct
team                                                  
PHI       11    4   0  0.733333        4    1   0  0.8
DAL       10    5   0  0.666667        4    1   0  0.8
NYG        5   10   0  0.333333        2    3   0  0.4
WAS        4   11   0  0.266667        0    5   0  0.0

In [12]:
# limit scope to teams still in contention
st = nfl(div).standings
teams = set(st[st[('overall','win')] >= st.iloc[0][('overall','win')] - len(weeks)].index)

# print the relevant schedule for reference. It's helpful to know
# who plays who
nfl.schedule(weeks, teams)

at hscore ascore
week ht                    
17   DAL  DET   None   None
     PHI  ARI   None   None
18   NYG  PHI   None   None
     WAS  DAL   None   None

In [13]:
# iterate over all possible outcomes and count the number
# of times each team wins the division.
# this can take 30 seconds or more in fast mode for a 2-week span

results = pd.DataFrame(columns=pd.MultiIndex.from_product([weeks, teams], names=['week','team']))
results.index.name = 'scenario'
results[('result','outcome')] = np.nan
results[('result','rule')] = np.nan

# calculate the number of scenarios to set up the progress bar
# it's the number of potential outcomes (win,loss.tie = 3)
# to the power of the number of relevant games (4 in this case)
# pow(3, 4) = 81

bar = IntProgress(min=0, max=3 ** len(nfl.schedule(weeks, teams)))
display(bar)

start = time.time()
for elem in nfl.scenarios(weeks, teams):
    nfl.clear(weeks)
    nfl.set(elem)

    t = nfl.tiebreaks(teams, fast=True)

    sch = nfl.schedule(teams, weeks)
    z = len(results)
    results.loc[z] = sch['wlt']
    results.loc[z, ('result','outcome')] = t.index[0]
    if len(t) > 1:
        results.loc[z, ('result','rule')] = t.iloc[1]
    else:
        results.loc[z, ('result','rule')] = 'overall'

    bar.value += 1

bar.layout.display = 'none'
print('Elapsed time: {}'.format(time.time() - start))
results

IntProgress(value=0, max=81)

Elapsed time: 19.21202778816223


week       17         18        result          
team      PHI  DAL   PHI   DAL outcome      rule
scenario                                        
0         win  win  loss  loss     PHI          
1         win  win  loss   win     DAL  division
2         win  win  loss   tie     PHI          
3         win  win   win  loss     PHI          
4         win  win   win   win     PHI          
...       ...  ...   ...   ...     ...       ...
76        tie  tie   win   win     PHI          
77        tie  tie   win   tie     PHI          
78        tie  tie   tie  loss     PHI          
79        tie  tie   tie   win     PHI          
80        tie  tie   tie   tie     PHI          

[81 rows x 6 columns]

In [11]:
# This shows how many scenarios result in each team winning the division
results.groupby(('result','outcome')).count()

week               17      18     result
team              PHI DAL PHI DAL   rule
(result, outcome)                       
DAL                15  15  15  15     15
PHI                66  66  66  66     66

In [12]:
# this will report all scenarios where the specified team wins
# their remaining games, answering the question whether they
# "control their own destiny." If so then the result will invariably
# be that the team wins the division, as shown in the 2nd column from the right

# Note that if a rule shows as conference|overall-rank or any form of netpoints,
# the analysis is not necessarily valid because the scenarios are based on
# outcomes only without specifying point spreads

results[(results.xs('DAL',level=1,axis=1) == 'win').all(axis=1)]

week        17         18       result            
team       PHI  DAL   PHI  DAL outcome        rule
scenario                                          
1          win  win  loss  win     DAL    division
4          win  win   win  win     PHI            
7          win  win   tie  win     PHI            
10        loss  win  loss  win     DAL            
13        loss  win   win  win     DAL  conference
16        loss  win   tie  win     DAL            
19         tie  win  loss  win     DAL            
22         tie  win   win  win     PHI            
25         tie  win   tie  win     DAL    division